In [1]:
import numpy as np

from a_nice_mc.models.discriminator import MLPDiscriminator
from a_nice_mc.models.generator import create_nice_network
from a_nice_mc.train.wgan_nll import Trainer

from hepmc.core.densities.qcd import ee_qq_ng
from hepmc.core.phase_space.mapping import MappedDensity
from hepmc.core.phase_space.rambo import RamboOnDiet

In [2]:
import tensorflow as tf
from a_nice_mc.objectives import Energy
from tensorflow.python.framework import ops

class QCDTarget(Energy):
    def __init__(self):
        super(QCDTarget, self).__init__()
        self.name = "QCDTarget"
        self.z = tf.placeholder(tf.float32, [None, 5], name='z')
        target = ee_qq_ng(1, 100., 5., .3)
        rambo_mapping = RamboOnDiet(100., 3)
        self.mapped = MappedDensity(target, rambo_mapping)
    
    def __call__(self, z):
        z1 = tf.reshape(tf.slice(z, [0, 0], [-1, 1]), [-1])
        z2 = tf.reshape(tf.slice(z, [0, 1], [-1, 1]), [-1])
        z3 = tf.reshape(tf.slice(z, [0, 2], [-1, 1]), [-1])
        z4 = tf.reshape(tf.slice(z, [0, 3], [-1, 1]), [-1])
        z5 = tf.reshape(tf.slice(z, [0, 4], [-1, 1]), [-1])
        return self.tf_energy(z1, z2, z3, z4, z5)
    
    # energy as numpy function
    def energy(self, z1, z2, z3, z4, z5):
        z = np.array([z1, z2, z3, z4, z5]).transpose()
        return self.mapped.pot(z).astype(np.float32, copy=False)
        #return self.camel.pot(z).view('float32')
    
    # energy as tensorflow function
    def tf_energy(self, z1, z2, z3, z4, z5, name=None):
        with tf.name_scope(name, "energy", [z1, z2, z3, z4, z5]) as name:
            y = tf.py_func(self.energy,
                       [z1, z2, z3, z4, z5],
                       [tf.float32],
                       name=name,
                       stateful=False)
            return y[0]
    
    #@staticmethod
    #def mean():
    #    return np.array(3*[.5])
    
    #@staticmethod
    #def std():
    #    return np.array(3*[.181])
    
    def evaluate(self, zv, path=None):
        pass

In [3]:
energy_fn = QCDTarget()

In [4]:
#def noise_sampler(bs):
#    return np.random.normal(0.0, 1.0, [bs, 5])

def noise_sampler(bs):
    noise = np.empty((bs, 5))
    
    indices = np.arange(bs)
    while indices.size > 0:
        x = np.random.normal(.5, .25, [indices.size, 5])
        y = energy_fn.mapped.pot(x)
        in_bounds = np.logical_not(np.isinf(y))
        noise[indices[in_bounds]] = x[in_bounds]
        indices = indices[np.logical_not(in_bounds)]

    return noise

In [5]:
discriminator = MLPDiscriminator([400, 400, 400])
generator = create_nice_network(
    5, 10,
    [
        ([400], 'v1', False),
        ([400, 400], 'x1', True),
        ([400], 'v2', False)
    ]
)

In [6]:
trainer = Trainer(generator, energy_fn, discriminator, noise_sampler, b=16, m=4)

In [7]:
trainer.train(max_iters=1000)

before bootstrap
before hmc_sampler.sample
Event 1	(batch acceptance rate: 0.000000)
Event 2	(batch acceptance rate: 0.000000)	max sequence length: 3
Event 3	(batch acceptance rate: 0.000000)	max sequence length: 4
Event 4	(batch acceptance rate: 0.000000)	max sequence length: 5
Event 5	(batch acceptance rate: 0.000000)	max sequence length: 6
Event 6	(batch acceptance rate: 0.000000)	max sequence length: 7
Event 7	(batch acceptance rate: 0.142857)	max sequence length: 7
Event 8	(batch acceptance rate: 0.125000)	max sequence length: 7
Event 9	(batch acceptance rate: 0.222222)	max sequence length: 7
Event 10	(batch acceptance rate: 0.200000)	max sequence length: 7
Event 20	(batch acceptance rate: 0.200000)	max sequence length: 7
Event 30	(batch acceptance rate: 0.200000)	max sequence length: 7
Event 40	(batch acceptance rate: 0.150000)	max sequence length: 15
Event 50	(batch acceptance rate: 0.160000)	max sequence length: 20
Event 60	(batch acceptance rate: 0.140000)	max sequence length:

[a_nice_mc.train.wgan_nll] [INFO] Iter [0] time [0.0000] d_loss [7.4880] g_loss [0.5506] v_loss [0.5219]
[a_nice_mc.train.wgan_nll] [INFO] Iter [100] time [19.0184] d_loss [-0.3114] g_loss [1.1841] v_loss [0.4850]
[a_nice_mc.train.wgan_nll] [INFO] Iter [200] time [36.8290] d_loss [-0.2981] g_loss [1.6151] v_loss [0.5239]
[a_nice_mc.train.wgan_nll] [INFO] Iter [300] time [54.4206] d_loss [-0.0181] g_loss [1.1971] v_loss [0.5347]
[a_nice_mc.train.wgan_nll] [INFO] Iter [400] time [71.9206] d_loss [-0.0677] g_loss [1.0840] v_loss [0.5224]
before bootstrap
[a_nice_mc.train.wgan_nll] [INFO] A-NICE-MC: batches [32] steps [6000 : 1] time [138.9138] samples/s [1382.1517]
before sample
[a_nice_mc.train.wgan_nll] [INFO] A-NICE-MC: batches [32] steps [6000 : 1] time [139.5430] samples/s [1375.9200]
z: [[[0.29687238 0.42126256 0.4408624  0.269585   0.70205796]
  [0.35197747 0.1607112  0.42195952 0.9512575  0.07716769]
  [0.35197747 0.1607112  0.42195952 0.9512575  0.07716769]
  ...
  [0.08130598 0.

In [17]:
sample = trainer.sample(batch_size=1, steps=100000)

[a_nice_mc.train.wgan_nll] [INFO] A-NICE-MC: batches [1] steps [100000 : 1] time [302.9317] samples/s [330.1074]


In [18]:
from hepmc.core.sampling import Sample
from hepmc.core.densities.qcd import export_hepmc
rambo_mapping = RamboOnDiet(100., 3)
anicemc_sample = Sample(data=rambo_mapping.map(sample[0][0]), weights=np.full(sample[0][0].shape[0], 1./sample[0][0].shape[0]))
export_hepmc(100., anicemc_sample, "../samples/qcd/2-3/a_nice_mc.hepmc")

In [24]:
sample_container = Sample(data=sample[0][0], target=target)

In [27]:
mean = sample_container.mean
variance = sample_container.variance

In [28]:
from hepmc.core.util.stat_tools import effective_sample_size

In [29]:
effective_sample_size(sample_container, mean, variance)

array([2377.57290738, 2676.03256435, 2816.47116257, 2162.44073713,
       2758.26628813])